In [ ]:
import keras


In [ ]:
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_83277f066db0464d8f10f2da52055039 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='V9Xl3fP-TKcSfGHqMtLrynQ-VgYBwtBi4klYMV42HeW2',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')


streaming_body_1 = client_83277f066db0464d8f10f2da52055039.get_object(Bucket='imageclassification-donotdelete-pr-esz9h2ilyklnkb', Key='Dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
if not hasattr(streaming_body_1,"__iter__"):streaming_body_1.__iter__=types.MethodType(__iter__,streaming_body_1)

In [ ]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)


In [ ]:
pwd

'/home/wsuser/work'

In [ ]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset/train_set')


In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 rotation_range=180,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
x_train =train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/train_set',
                                         target_size = (128 ,128),
                                         batch_size = 32 ,
                                         class_mode = 'binary')
                                  

Found 436 images belonging to 2 classes.


In [ ]:
x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/test_set',
                                          target_size = (128 ,128),
                                          batch_size = 32 ,
                                          class_mode = 'binary')

Found 121 images belonging to 2 classes.


Model Building

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#initialize the model
model=Sequential()

In [ ]:
from keras.layers.pooling.max_pooling2d import MaxPooling2D
#add convolution layer
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add flatten layer
model.add(Flatten())

In [ ]:
#add hidden layer
model.add(Dense(150,activation='relu'))
#add output layer
model.add(Dense(1,activation='sigmoid'))

In [ ]:
#configure the learning process
model.compile(loss = 'binary_crossentropy',
              optimizer = "adam",
              metrics = ["accuracy"])

In [ ]:
#training the model
model.fit(x_train,steps_per_epoch=14,
                    epochs=10,validation_data=x_test,
                    validation_steps=4)

Epoch 1/10
14/14 [==============================] - 32s 2s/step - loss: 6.6102 - accuracy: 0.5413 - val_loss: 0.6909 - val_accuracy: 0.6281
Epoch 2/10
14/14 [==============================] - 30s 2s/step - loss: 0.6504 - accuracy: 0.6972 - val_loss: 0.2128 - val_accuracy: 0.9256
Epoch 3/10
14/14 [==============================] - 28s 2s/step - loss: 0.3389 - accuracy: 0.8349 - val_loss: 0.1659 - val_accuracy: 0.9339
Epoch 4/10
14/14 [==============================] - 26s 2s/step - loss: 0.2783 - accuracy: 0.8830 - val_loss: 0.1539 - val_accuracy: 0.9504
Epoch 5/10
14/14 [==============================] - 31s 2s/step - loss: 0.2055 - accuracy: 0.9174 - val_loss: 0.1590 - val_accuracy: 0.9421
Epoch 6/10
14/14 [==============================] - 28s 2s/step - loss: 0.2063 - accuracy: 0.9083 - val_loss: 0.1136 - val_accuracy: 0.9504
Epoch 7/10
14/14 [==============================] - 28s 2s/step - loss: 0.1874 - accuracy: 0.9266 - val_loss: 0.1030 - val_accuracy: 0.9752
Epoch 8/10
14/14 [==

In [ ]:
#save the model
model.save("forest1.h5")

In [ ]:
!tar -zcvf image-classification_new.tgz forest.h5


In [ ]:
ls  -1

Dataset/
image-classification_new.tgz


In [ ]:
!pip install watson-machine-learning-client --upgrade


     |████████████████████████████████| 538 kB 12.5 MB/s eta 0:00:01


In [ ]:
import ibm_watson_machine_learning
from ibm_watson_machine_learning import APIClient
wml_credentilas = {
                   "url":"https://us-south.ml.cloud.ibm.com",
                   "apikey":"hxe6koyIaU12_be6Qw-sQ8omzOrg9czDp9Ep11YppBs6"
}
client = APIClient(wml_credentilas)


In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client, 'Forestrecognition')
print("Space UID = " + space_uid)


Space UID = 2bae4b0b-57cd-4fd3-89ef-5fc4a44867a5


In [ ]:
client.set.default_space(space_uid)


'SUCCESS'

In [ ]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid


'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [ ]:
model_details = client.repository.store_model(model='image-classification_new.tgz',meta_props={ 
client.repository.ModelMetaNames.NAME:'CNN',
client.repository.ModelMetaNames.TYPE:"tensorflow_rt22.1",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)


This method is deprecated, please use get_model_id()


In [ ]:
model_id

'f85fe154-cc0d-45cd-a597-bf36e5814ae0'

In [ ]:
#import load_model from keras.model
from keras.models import load_model
#import image class from keras
from tensorflow.keras.preprocessing import image
#import numpy
import numpy as np
#import cv2
import cv2

In [ ]:
#load the saved model
model = load_model("forest1.h5")

In [ ]:
img=image.load_img(r'/home/wsuser/work/Dataset/test_set/with fire/180802_CarrFire_010_large_700x467.jpg')
x=image.img_to_array(img)
res = cv2.resize(x, dsize=(128,128),interpolation=cv2.INTER_CUBIC)
x=np.expand_dims(res,axis=0)

In [ ]:
pred = model.predict(x)

1/1 [==============================] - 0s 308ms/step


In [ ]:
pred

array([[1.]], dtype=float32)

OpenCV for Vedio Processing

In [ ]:
pip install twilio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 4.7 MB/s 


In [ ]:
pip install playsound

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7035 sha256=2675bb34916410c243ebfa7ebedbf355c6bd1f40538969d83c90042d0ed07ae0
  Stored in directory: /root/.cache/pip/wheels/ba/f8/bb/ea57c0146b664dca3a0ada4199b0ecb5f9dfcb7b7e22b65ba2
Successfully built playsound


In [ ]:
from logging import WARNING
#import opencv library
import cv2
#import numpy
import numpy as np
#import image function from keras
from keras.preprocessing import image
#import load_model from keras
from keras.models import load_model
#import client from twilio API
from twilio.rest import Client
#import playsound package
from playsound import playsound


In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_83277f066db0464d8f10f2da52055039 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='V9Xl3fP-TKcSfGHqMtLrynQ-VgYBwtBi4klYMV42HeW2',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')


streaming_body_1 = client_83277f066db0464d8f10f2da52055039.get_object(Bucket='imageclassification-donotdelete-pr-esz9h2ilyklnkb', Key='Dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
if not hasattr(streaming_body_1,"__iter__"):streaming_body_1.__iter__=types.MethodType(__iter__,streaming_body_1)

In [ ]:
from twilio.rest import Client
from playsound import playsound
if pred==0:
  print('Forest fire')
  account_sid='ACe23d3104e6d355111543395761f7371a'
  auth_token='48801591e5794a1fc8cdd69e928cf77f'
  client=Client(account_sid,auth_token)
  message=client.messages \
  .create(
      body='forest fire is detected,stay alert',
      #use twilio free number
      from_='+14254751445',
      #to number
      to='+919080336426')
  print(message.sid)
  print("Fire detected")
  print("SMS Sent!")
elif pred==1:
  print('No danger')


No danger
